*****
## WFS Layer Extract
*****
Author: Mackenzie Rock
Date: June 3, 2025
Goal: The goal of this Jupyter notebook is to determine a foundation for extracting the relevant WFS layers identified and storing them in a suitable format for uploading to the postgreSQL database. In this section I will test:
- The code to extract
- I will visualize the data to ensure I have capture it correctly
- I will test several samples overtime
- Transformation into suitable format for load

### Extraction Code

To be extracted from WFS:
- Fire Danger (PUBLIC:FDR_CURRENT_SHP)
- Fire Perimeter Estimate (PUBLIC:M3_POLYGONS_CURRENT)
- Fire M3 Hotspots (PUBLIC:HOTSPOTS_LAST24HRS)
- Season-to-date Hotspots (TBD)
- Active Fires (PUBLIC:ACTIVEFIRES_CURRENT)
- Forecast Weather Stations (PUBLIC:FIREWX_STNS & PUBLIC:FIREWX_STNS_CURRENT)
- Reporting Weather Stations (PUBLIC:FIREWX_SCRIBE & PUBLIC:FIREWX_SCRIBE_FCST)
- Fire History (PUBLIC:REPORTEDFIRES_2024 & PUBLIC:REPORTEDFIRES_YTD)
- Check on what PUBLIC:BASEMAP_INSIDE_BNDRY & PUBLIC:BASEMAP_LAND are

In [19]:
import geopandas as gpd
import folium
from shapely.geometry import mapping
import os
import requests
from io import BytesIO
import pandas as pd

# WFS endpoint
WFS_URL = "https://cwfis.cfs.nrcan.gc.ca/geoserver/public/ows"

# Output directory
OUTPUT_DIR = "./wfs_layers"

if os.path.exists(OUTPUT_DIR):
    pass
else:
    os.makedirs(OUTPUT_DIR, exist_ok=True)



def fetch_and_visualize_wfs_layer(layer_name: str, date: str = None, output_format: str = "GeoJSON"):
    """
    Fetches a layer from the CWFIS WFS endpoint, reprojects to EPSG:4326,
    saves to file, and visualizes using Folium.
    """
    # Build WFS URL
    params = {
        "service": "WFS",
        "version": "1.0.0",
        "request": "GetFeature",
        "typename": layer_name,
        "outputFormat": "application/json"
    }
    query_url = f"{WFS_URL}?{'&'.join([f'{k}={v}' for k, v in params.items()])}"

    print(f"Fetching {layer_name} from WFS...")


    # Read GeoDataFrame
    response = requests.get(query_url)
    response.raise_for_status()  # Fail early if bad response
    print(response.content)
    gdf = gpd.read_file(BytesIO(response.content))

    # Reproject if needed
    if gdf.crs and gdf.crs.to_string() != "EPSG:4326":
        gdf = gdf.to_crs("EPSG:4326")

    # Construct filename
    filename = f"{layer_name.replace(':', '_')}_{date if date else 'latest'}.{output_format.lower()}"
    filepath = os.path.join(OUTPUT_DIR, filename)

    # Save to file
    if output_format.lower() == "geojson":
        gdf.to_file(filepath, driver="GeoJSON")
    elif output_format.lower() == "gpkg":
        gdf.to_file(filepath, driver="GPKG")
    else:
        raise ValueError("Unsupported output format")


    # Visualize with Folium
    if not gdf.empty:
        centroid = gdf.geometry.centroid.iloc[0]
        fmap = folium.Map(location=[centroid.y, centroid.x], zoom_start=5)

        json_safe_gdf = gdf.copy()
        for col in json_safe_gdf.columns:
            if pd.api.types.is_datetime64_any_dtype(json_safe_gdf[col]) or isinstance(json_safe_gdf[col].iloc[0], pd.Timestamp):
                print(f"Converting datetime column '{col}' to string")
                json_safe_gdf[col] = json_safe_gdf[col].astype(str)

        folium.GeoJson(json_safe_gdf).add_to(fmap)
    else:
        fmap = folium.Map(location=[56, -106], zoom_start=4)  # fallback

    return gdf, filepath, fmap

In [20]:
#from wfs_utils import fetch_and_visualize_wfs_layer

map_output_dir = "./maps"
layer_names = {
    "Fire_Danger": "public:fdr_current_shp",
    "Fire_Perimeter_Estimate": "public:m3_polygons_current",
    "M3_Hotspots": "public:hotspots_last24hrs",
    "Active_Fires": "public:activefires_current",
    "Forecast_Weather_Stations": "public:firewx_stns",
    "Forecast_Weather_Stations_Current": "public:firewx_stns_current",
    "Reporting_Weather_Stations": "public:firewx_scribe",
    "Reporting_Weather_Stations_Forecast": "public:firewx_scribe_fcst",
    "Fire_History_YTD": "public:reportedfires_ytd",
    "Fire_History_2024": "public:reportedfires_2024"
}
date = "2024-01-01"



for label, type in layer_names.items():

    print(f"Attempting to process layer: {label}")
    gdf, path, fmap = fetch_and_visualize_wfs_layer(type, date)
    map_path = os.path.join(map_output_dir, f"{label}_{date}.html")
    fmap.save(map_path)





Attempting to process layer: Fire_Danger
Fetching public:fdr_current_shp from WFS...


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

/tmp/ipykernel_402254/1394227952.py:65: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = gdf.geometry.centroid.iloc[0]


Attempting to process layer: Fire_Perimeter_Estimate
Fetching public:m3_polygons_current from WFS...


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

/tmp/ipykernel_402254/1394227952.py:65: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = gdf.geometry.centroid.iloc[0]


Converting datetime column 'firstdate' to string
Converting datetime column 'lastdate' to string
Attempting to process layer: M3_Hotspots
Fetching public:hotspots_last24hrs from WFS...


...

/tmp/ipykernel_402254/1394227952.py:65: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = gdf.geometry.centroid.iloc[0]


Attempting to process layer: Reporting_Weather_Stations
Fetching public:firewx_scribe from WFS...
b'{"type":"FeatureCollection","features":[{"type":"Feature","id":"firewx_scribe.HAX","geometry":{"type":"Point","coordinates":[-1760659.32460629,391933.90767241]},"geometry_name":"the_geom","properties":{"wmo":75345,"name":"APEX","rep_date":"2025-06-03T12:00:00Z","latitude":49.38333333,"longitude":-119.9166667,"elevation":1875,"temp":17,"rh":28.333333,"ws":4.333177,"wdir":283.615387,"precip":0,"sog":0,"ffmc":90.486524,"dmc":37.962061,"dc":418.889598,"isi":5.71798,"bui":61.899863,"fwi":16.543121,"dsr":3.904108,"wx":4.211405,"wy":-1.020043,"timezone":8,"x":-1760659.32460629,"y":391933.907672414}},{"type":"Feature","id":"firewx_scribe.WAR","geometry":{"type":"Point","coordinates":[2910534.00862252,782269.03760323]},"geometry_name":"the_geom","properties":{"wmo":71807,"name":"ARGENTIA","rep_date":"2025-06-03T12:00:00Z","latitude":47.3,"longitude":-54,"elevation":16,"temp":10,"rh":67.666667,"ws

/tmp/ipykernel_402254/1394227952.py:65: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = gdf.geometry.centroid.iloc[0]


Attempting to process layer: Reporting_Weather_Stations_Forecast
Fetching public:firewx_scribe_fcst from WFS...


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

/tmp/ipykernel_402254/1394227952.py:65: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = gdf.geometry.centroid.iloc[0]
